In [1]:
import pandas as pd

In [2]:
import os

In [3]:
os.chdir('E:/Learning_Spoons_Second_Project')

In [4]:
#데이터 불러오기
df_sun = pd.read_csv('data/일조량.csv')
df_tomato = pd.read_csv('data/토마토_도매가격.csv')
df_time = pd.read_csv('data/날씨_시계열.csv',encoding='euc-kr')

In [5]:
#머신러닝을 위해 도를 int 형식으로 변환을 위한 dictionary
dict_1 = {'강원도':0, '경기도':1, '인천광역시':2, '서울특별시':3, '경상북도':4, '충청북도':5, '충청남도':6, '대전광역시':7,
       '전라북도':8, '대구광역시':9, '울산광역시':10, '경상남도':11, '광주광역시':12, '부산광역시':13, '전라남도':14,
       '제주특별자치도':15}

In [6]:
#도_num 컬럼을 dict_1을 이용하여 생성
df_tomato['도_num'] = df_tomato['도'].map(dict_1)
df_sun['도_num'] = df_sun['도'].map(dict_1)
df_time['도_num'] = df_time['도'].map(dict_1)

In [7]:
#세 데이터 merge를 위해 일시 및 도_num으로 groupby하여 평균값 계산
df_s = df_sun.groupby(['일시','도_num']).mean().reset_index()

In [8]:
#2013년 및 2014년 pm25는 측정값이 없기 때문에 날짜 컬럼을 생성하여 날짜에 따른 PM25 평균값으로 대체 

df_time[['월','일']] = df_time[['월','일']].astype('str') 

#null값 처리를 위해 날짜 컬럼 생성
df_time['날짜'] = df_time['월'] + '-' + df_time['일']

#날짜별 PM25 평균을 구해서 df_mean에 담기
df_mean = df_time[~df_time['PM25'].isnull()].groupby('날짜')['PM25'].mean().reset_index()

#df_mean 제대로 생성되었는지 확인
df_mean

#날짜를 키로 PM25를 value로 딕셔러니 생성
fill_na = dict(zip(df_mean['날짜'],df_mean['PM25']))

#PM25의 null값들을 dictionary value로 채움
df_time.loc[df_time['PM25'].isnull(), 'PM25'] = df_time['날짜'].map(fill_na)

In [9]:
# merge를 위해 날짜 컬럼 drop
df_time = df_time.drop('날짜',axis=1)

In [10]:
#merge를 위해 다른 두 데이터에는 없는 세종시를 충청남도에 포함
df_time.loc[df_time['도'].str.contains('세종'),'도'] = '충청남도'

In [11]:
#merge를 위해 일시 및 도_num으로 groupby하여 평균값 계산
df_t=df_time.groupby(['일시','도_num']).mean().reset_index()

In [12]:
#groupby 되었는지 확인
df_t

,일시,도_num,연도,SO2,CO,O3,NO2,PM10,PM25
0,2013-01-01,0.0,2013,0.007216,0.770076,0.020583,0.018420,45.132576,31.543635
1,2013-01-01,1.0,2013,0.006340,0.649199,0.013143,0.031923,44.532051,31.543635
2,2013-01-01,2.0,2013,0.007477,0.713492,0.013913,0.030154,39.626984,31.543635
3,2013-01-01,3.0,2013,0.005741,0.658125,0.008458,0.043541,38.334375,31.543635
4,2013-01-01,4.0,2013,0.006363,0.739583,0.015653,0.017980,48.280093,31.543635
...,...,...,...,...,...,...,...,...,...
32860,2018-12-31,10.0,2018,0.003241,0.502206,0.018193,0.023311,22.787681,12.066176
32861,2018-12-31,11.0,2018,0.003150,0.475868,0.017313,0.022639,29.648866,19.518839
32862,2018-12-31,12.0,2018,0.003204,0.605093,0.015468,0.027125,34.759259,22.728704
32863,2018-12-31,13.0,2018,0.004119,0.373418,0.014718,0.027532,28.525591,15.960829


In [13]:
#merge를 위해 월,일 컬럼 drop
df_s = df_s.drop(['월','일'],axis=1)

In [14]:
#drop 되었는지 확인
df_s

,일시,도_num,연도,합계 일조시간(hr)
0,2013-01-01,0,2013,3.115385
1,2013-01-01,1,2013,3.800000
2,2013-01-01,2,2013,5.533333
3,2013-01-01,3,2013,4.600000
4,2013-01-01,4,2013,2.686667
...,...,...,...,...
35051,2018-12-31,11,2018,5.921429
35052,2018-12-31,12,2018,9.200000
35053,2018-12-31,13,2018,5.700000
35054,2018-12-31,14,2018,8.471429


In [15]:
#일조량 데이터와 대기오염 데이터 merge
df_air = pd.merge(df_t,df_s,on=['일시','도_num','연도'])

In [16]:
#데이터 타입 확인
df_air.dtypes

일시              object
도_num          float64
연도               int64
SO2            float64
CO             float64
O3             float64
NO2            float64
PM10           float64
PM25           float64
합계 일조시간(hr)    float64
dtype: object

In [17]:
#merge가 맞게 되었는지 확인
df_air

,일시,도_num,연도,SO2,CO,O3,NO2,PM10,PM25,합계 일조시간(hr)
0,2013-01-01,0.0,2013,0.007216,0.770076,0.020583,0.018420,45.132576,31.543635,3.115385
1,2013-01-01,1.0,2013,0.006340,0.649199,0.013143,0.031923,44.532051,31.543635,3.800000
2,2013-01-01,2.0,2013,0.007477,0.713492,0.013913,0.030154,39.626984,31.543635,5.533333
3,2013-01-01,3.0,2013,0.005741,0.658125,0.008458,0.043541,38.334375,31.543635,4.600000
4,2013-01-01,4.0,2013,0.006363,0.739583,0.015653,0.017980,48.280093,31.543635,2.686667
...,...,...,...,...,...,...,...,...,...,...
32860,2018-12-31,10.0,2018,0.003241,0.502206,0.018193,0.023311,22.787681,12.066176,6.600000
32861,2018-12-31,11.0,2018,0.003150,0.475868,0.017313,0.022639,29.648866,19.518839,5.921429
32862,2018-12-31,12.0,2018,0.003204,0.605093,0.015468,0.027125,34.759259,22.728704,9.200000
32863,2018-12-31,13.0,2018,0.004119,0.373418,0.014718,0.027532,28.525591,15.960829,5.700000


In [18]:
#토마토 가격데이터와 기상 데이터 merge (최종 데이터)
df_tomatoma = pd.merge(df_air,df_tomato,on=['일시','도_num','연도'],how='left')

In [22]:
df_tomatoma[~df_tomatoma['일시'].str.contains('2013|2014|2018')]

,일시,도_num,연도,SO2,CO,O3,NO2,PM10,PM25,합계 일조시간(hr),품종명,도,마켓명,날짜,가격,월,일,연월
10950,2015-01-01,0.0,2015,0.004804,0.523007,0.023921,0.009153,65.045455,24.020192,7.930769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10951,2015-01-01,1.0,2015,0.004798,0.411294,0.020143,0.011644,69.002664,28.909265,8.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10952,2015-01-01,2.0,2015,0.004998,0.408929,0.021345,0.009187,65.023810,27.846832,6.266667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10953,2015-01-01,3.0,2015,0.005268,0.386432,0.017600,0.014946,65.684411,19.724415,8.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10954,2015-01-01,4.0,2015,0.004866,0.426097,0.026598,0.005901,42.131020,31.543635,7.886667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27385,2017-12-31,10.0,2017,0.003583,0.700260,0.028560,0.017779,69.770833,34.403416,8.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27386,2017-12-31,11.0,2017,0.003392,0.557803,0.029622,0.013610,71.634133,31.628244,7.985714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27387,2017-12-31,12.0,2017,0.002731,0.600926,0.027796,0.016380,79.917069,31.310185,6.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27388,2017-12-31,13.0,2017,0.003782,0.399265,0.028774,0.014218,78.670238,38.210317,8.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#품종명은 토마토 밖에 없기때문에 토마토(10kg)로 nan값 채우기
df_tomatoma['품종명'] = df_tomatoma['품종명'].fillna('토마토(10kg)')

In [20]:
#위에 생성된 dictionary의 key값과 value값 변경
dict_reversed = {v: k for k, v in dict_1.items()}

In [21]:
#float 타입으로 되어있는 도_num int값으로 변경
df_tomatoma['도_num'] = df_tomatoma['도_num'].astype('int')

In [22]:
#dict_reverse를 이용하여 도 컬럼 생성
df_tomatoma.loc[df_tomatoma['도'].isnull(),'도'] = df_tomatoma['도_num'].map(dict_reversed)

In [23]:
#가격이 없는 일시는 장이 안선 날이기때문에 가격이 null아닌 일시만 list로 변환
change_list = list(df_tomatoma.loc[~df_tomatoma['가격'].isnull(),'일시'].unique())

In [24]:
# list에 있는 값 확인
change_list

['2013-01-02',
 '2013-01-03',
 '2013-01-04',
 '2013-01-07',
 '2013-01-08',
 '2013-01-09',
 '2013-01-10',
 '2013-01-11',
 '2013-01-14',
 '2013-01-15',
 '2013-01-16',
 '2013-01-17',
 '2013-01-18',
 '2013-01-21',
 '2013-01-22',
 '2013-01-23',
 '2013-01-24',
 '2013-01-25',
 '2013-01-28',
 '2013-01-29',
 '2013-01-30',
 '2013-01-31',
 '2013-02-01',
 '2013-02-04',
 '2013-02-05',
 '2013-02-06',
 '2013-02-07',
 '2013-02-08',
 '2013-02-12',
 '2013-02-13',
 '2013-02-14',
 '2013-02-15',
 '2013-02-18',
 '2013-02-19',
 '2013-02-20',
 '2013-02-21',
 '2013-02-22',
 '2013-02-25',
 '2013-02-26',
 '2013-02-27',
 '2013-02-28',
 '2013-03-04',
 '2013-03-05',
 '2013-03-06',
 '2013-03-07',
 '2013-03-08',
 '2013-03-11',
 '2013-03-12',
 '2013-03-13',
 '2013-03-14',
 '2013-03-15',
 '2013-03-18',
 '2013-03-19',
 '2013-03-20',
 '2013-03-21',
 '2013-03-22',
 '2013-03-25',
 '2013-03-26',
 '2013-03-27',
 '2013-03-28',
 '2013-03-29',
 '2013-04-01',
 '2013-04-02',
 '2013-04-03',
 '2013-04-04',
 '2013-04-05',
 '2013-04-

In [26]:
#change_list에 있는 일시만으로 새로운 데이터 프레임 생성
#df_tomatoma_final = df_tomatoma[df_tomatoma['일시'].isin(change_list)].copy()

In [48]:
df_tomatoma_final = df_tomatoma.copy()

In [49]:
#현재 안 쓸 컬럼들 드랍
df_tomatoma_final = df_tomatoma_final.drop(['날짜','월','일','연월'],axis=1)

In [50]:
#가격이 nan인 값들을 권역을 설정하여 채워주기 위해 리스트 생성
광주_lst = ['광주광역시','전라북도','전라남도','제주도']
대구_lst = ['대구광역시','경상북도']
부산_lst = ['부산광역시','경상남도','울산광역시']
대전_lst = ['대전광역시','충청북도','충청남도']
서울_lst = ['서울특별시','경기도','강원도','인천광역시']

In [51]:
#nan값을 채우기 위해 interpolate의  time method를 사용하기 위해 일시를 date type으로 변경
df_tomatoma_final['일시']=pd.to_datetime(df_tomatoma_final['일시'])

In [52]:
#일시를 index로 변경
df_tomatoma_final = df_tomatoma_final.set_index('일시')

In [53]:
#변경이 되었는지 확인
df_tomatoma_final

,도_num,연도,SO2,CO,O3,NO2,PM10,PM25,합계 일조시간(hr),품종명,도,마켓명,가격
일시,,,,,,,,,,,,,
2013-01-01,0,2013,0.007216,0.770076,0.020583,0.018420,45.132576,31.543635,3.115385,토마토(10kg),강원도,NaN,NaN
2013-01-01,1,2013,0.006340,0.649199,0.013143,0.031923,44.532051,31.543635,3.800000,토마토(10kg),경기도,NaN,NaN
2013-01-01,2,2013,0.007477,0.713492,0.013913,0.030154,39.626984,31.543635,5.533333,토마토(10kg),인천광역시,NaN,NaN
2013-01-01,3,2013,0.005741,0.658125,0.008458,0.043541,38.334375,31.543635,4.600000,토마토(10kg),서울특별시,NaN,NaN
2013-01-01,4,2013,0.006363,0.739583,0.015653,0.017980,48.280093,31.543635,2.686667,토마토(10kg),경상북도,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31,10,2018,0.003241,0.502206,0.018193,0.023311,22.787681,12.066176,6.600000,토마토(10kg),울산광역시,NaN,NaN
2018-12-31,11,2018,0.003150,0.475868,0.017313,0.022639,29.648866,19.518839,5.921429,토마토(10kg),경상남도,NaN,NaN
2018-12-31,12,2018,0.003204,0.605093,0.015468,0.027125,34.759259,22.728704,9.200000,토마토(10kg),광주광역시,각화도매,31000.0


In [54]:
#각 권역의 마켓명이 같을 것이라고 가정하여 fillna의 bfill을 사용하여 마켓명 채우기위한 함수
def fill_market(lst):
    df_tomatoma_final.loc[df_tomatoma_final['도'].isin(lst),'마켓명'] =  df_tomatoma_final.loc[df_tomatoma_final['도'].isin(lst),'마켓명'].fillna(method='bfill')
    return df_tomatoma_final.loc[df_tomatoma_final['도'].isin(lst),'마켓명']

In [55]:
#함수를 활용하여 마켓명 nan값 처리
fill_market(광주_lst)
fill_market(대구_lst)
fill_market(부산_lst)
fill_market(대전_lst)
fill_market(서울_lst)

일시
2013-01-01    가락도매
2013-01-01    가락도매
2013-01-01    가락도매
2013-01-01    가락도매
2013-01-02    가락도매
              ... 
2018-12-30    가락도매
2018-12-31    가락도매
2018-12-31    가락도매
2018-12-31    가락도매
2018-12-31    가락도매
Name: 마켓명, Length: 8764, dtype: object

In [61]:
#함수를 사용하였지만 값이 안들어간 마켓명이 존재하여 하드코딩으로 값 채움
df_tomatoma_final.loc[df_tomatoma_final['마켓명'].isnull(),'마켓명'] = '각화도매'

In [82]:
market_lst

['가락도매', '북부도매', '오정도매', '각화도매', '엄궁도매']

In [83]:
#마켓별로 분할
market_lst = list(df_tomatoma_final['마켓명'].unique())
def divide_market(lst):
    for i in range(len(lst)):
        globals()['df_{}'.format(lst[i])] = df_tomatoma_final[df_tomatoma_final['마켓명'].str.contains(lst[i])]
        print(lst[i])
    return globals()['df_{}'.format(lst[i])]

In [91]:
def fill_price(data_frame):
    data_frame.loc[~data_frame.index.isin(change_list),'가격'] = -1
    data_frame.loc[data_frame.index.isin(change_list),'가격'] = data_frame.loc[data_frame.index.isin(change_list),'가격'].interpolate(method='time',limit_direction='both')
    return data_frame

In [92]:
fill_price(df_가락도매)

C:\Users\LG\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,도_num,연도,SO2,CO,O3,NO2,PM10,PM25,합계 일조시간(hr),품종명,도,마켓명,가격
일시,,,,,,,,,,,,,
2013-01-01,0,2013,0.007216,0.770076,0.020583,0.018420,45.132576,31.543635,3.115385,토마토(10kg),강원도,가락도매,-1.0
2013-01-01,1,2013,0.006340,0.649199,0.013143,0.031923,44.532051,31.543635,3.800000,토마토(10kg),경기도,가락도매,-1.0
2013-01-01,2,2013,0.007477,0.713492,0.013913,0.030154,39.626984,31.543635,5.533333,토마토(10kg),인천광역시,가락도매,-1.0
2013-01-01,3,2013,0.005741,0.658125,0.008458,0.043541,38.334375,31.543635,4.600000,토마토(10kg),서울특별시,가락도매,-1.0
2013-01-02,0,2013,0.005504,0.503030,0.030148,0.009233,35.018939,37.602906,8.300000,토마토(10kg),강원도,가락도매,29000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-30,3,2018,0.004657,0.639330,0.011581,0.031490,32.668705,16.896871,8.700000,토마토(10kg),서울특별시,가락도매,-1.0
2018-12-31,0,2018,0.003162,0.623345,0.015092,0.019268,34.180676,26.822842,7.671429,토마토(10kg),강원도,가락도매,25000.0
2018-12-31,1,2018,0.004220,0.772437,0.008614,0.039955,51.498857,32.527694,7.880000,토마토(10kg),경기도,가락도매,25000.0


In [38]:
import os
os.getcwd()

'E:\\Learning_Spoons_Second_Project'

In [39]:
#df_tomatoma_final.to_csv('data/full_data.csv')

In [42]:
#pd.read_csv('data/full_data.csv')

,일시,도_num,연도,SO2,CO,O3,NO2,PM10,PM25,합계 일조시간(hr),품종명,도,마켓명,가격
0,2013-01-02,0,2013,0.005504,0.503030,0.030148,0.009233,35.018939,37.602906,8.300000,토마토(10kg),강원도,가락도매,29000.0
1,2013-01-02,1,2013,0.005632,0.417254,0.023083,0.018736,24.678419,37.602906,8.820000,토마토(10kg),경기도,가락도매,29000.0
2,2013-01-02,2,2013,0.006906,0.467460,0.024183,0.014894,24.450397,37.602906,6.133333,토마토(10kg),인천광역시,가락도매,29000.0
3,2013-01-02,3,2013,0.004950,0.403021,0.024425,0.021413,18.644792,37.602906,9.100000,토마토(10kg),서울특별시,가락도매,29000.0
4,2013-01-02,4,2013,0.005591,0.567593,0.025477,0.011117,44.833333,37.602906,7.980000,토마토(10kg),경상북도,북부도매,29000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22075,2018-12-31,10,2018,0.003241,0.502206,0.018193,0.023311,22.787681,12.066176,6.600000,토마토(10kg),울산광역시,엄궁도매,30000.0
22076,2018-12-31,11,2018,0.003150,0.475868,0.017313,0.022639,29.648866,19.518839,5.921429,토마토(10kg),경상남도,엄궁도매,30000.0
22077,2018-12-31,12,2018,0.003204,0.605093,0.015468,0.027125,34.759259,22.728704,9.200000,토마토(10kg),광주광역시,각화도매,31000.0
22078,2018-12-31,13,2018,0.004119,0.373418,0.014718,0.027532,28.525591,15.960829,5.700000,토마토(10kg),부산광역시,엄궁도매,30000.0
